In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). 

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;

In [ ]:
df_transactions = pd.read_csv("drive/MyDrive/ML/transactions.csv", nrows=1000000, delimiter=',')
df_gender = pd.read_csv("drive/MyDrive/ML/gender_train.csv", delimiter=',')
df_mcc_codes = pd.read_csv("drive/MyDrive/ML/tr_mcc_codes.csv", delimiter=';')
df_types = pd.read_csv("drive/MyDrive/ML/tr_types.csv", delimiter=';')



Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [ ]:
transaction_sample = df_transactions.sample(n=1000)
print(transaction_sample.head())

pos_or_atm_tr_types_set = set(code for code in df_types.loc[df_types['tr_description'].str.contains("АТМ|POS")].tr_type)

pos_or_atm_tr_types_counter = 0
for tr in transaction_sample.mcc_code:
  if tr in pos_or_atm_tr_types_set:
    pos_or_atm_tr_types_counter += 1

print(f"\nДоля транзакций: {(pos_or_atm_tr_types_counter/len(transaction_sample)*100):.2f}%")

        customer_id   tr_datetime  mcc_code  tr_type    amount term_id
900011     56436494  116 20:05:13      6011     2010 -13475.49     NaN
86942      39193827  281 14:31:54      4814     1030  -1122.96  154949
446207     70780820  377 05:29:49      6010     7070  22818.50  888987
455921     70780820  432 10:45:29      6010     7070   2919.69  888912
428161     70780820  280 03:59:47      6010     7070  26434.43  888986

Доля транзакций: 20.60%


## Задание 2 


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [ ]:
n = 10
top_10_full = df_transactions['tr_type'].value_counts()[:n]
top_10 = top_10_full.index.tolist()

print("Сколько раз встретился | Код | Описание")
for tr_type in top_10:
  print(f"{top_10_full[tr_type]}\t|\t{tr_type}\t|\t{df_types.loc[df_types['tr_type'] == tr_type].iloc[0]['tr_description']}")

print(top_10_full)

print('============================== Доп. задание ================================')
df_tr_with_genders = pd.merge(df_transactions, df_gender, how='left')
df_all_mean = df_tr_with_genders.loc[df_tr_with_genders['tr_type'].isin(top_10)].loc[df_transactions['amount'] < 0].groupby('tr_type')['amount'].mean().reset_index(name ='amount')
df_0_mean = df_tr_with_genders.loc[df_tr_with_genders['tr_type'].isin(top_10)].loc[df_tr_with_genders['gender'] == 0].loc[df_transactions['amount'] < 0].groupby('tr_type')['amount'].mean().reset_index(name ='gender_0_amount_mean')
df_1_mean = df_tr_with_genders.loc[df_tr_with_genders['tr_type'].isin(top_10)].loc[df_tr_with_genders['gender'] == 1].loc[df_transactions['amount'] < 0].groupby('tr_type')['amount'].mean().reset_index(name ='gender_1_amount_mean')

df_total = pd.DataFrame()
df_total['tr_type'] = df_0_mean['tr_type']
df_total['c0'] = df_0_mean['gender_0_amount_mean']/df_all_mean['amount']
df_total['c1'] = df_1_mean['gender_1_amount_mean']/df_all_mean['amount']

df_total['max_val'] = df_total[["c0", "c1"]].max(axis=1)
df_total.sort_values("max_val", ascending=False, inplace=True)

print(df_total.drop('max_val', axis=1))

Сколько раз встретился | Код | Описание
231117	|	1010	|	Покупка. POS ТУ СБ РФ
151166	|	2010	|	Выдача наличных в АТМ Сбербанк России
149006	|	7070	|	Перевод на карту (с карты) через Мобильный банк (без взимания комиссии с отправителя)
137658	|	1110	|	Покупка. POS ТУ Россия
118975	|	1030	|	Оплата услуги. Банкоматы СБ РФ
49830	|	2370	|	Списание с карты на карту по операции <перевода с карты на карту> через Мобильный банк (без комиссии)
28972	|	7010	|	Взнос наличных через АТМ (в своем тер.банке)
26078	|	7030	|	Перевод на карту (с карты) через АТМ (в пределах одного тер.банка)
15781	|	7071	|	Перевод на карту (с карты) через Мобильный банк (с взиманием комиссии с отправителя)
15476	|	1100	|	Покупка. ТУ  Россия
1010    231117
2010    151166
7070    149006
1110    137658
1030    118975
2370     49830
7010     28972
7030     26078
7071     15781
1100     15476
Name: tr_type, dtype: int64
============================== Доп. задание ================================
   tr_type        c0        c1



## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [ ]:
most_incoming = df_transactions \
                    .loc[df_transactions['amount'] > 0] \
                    .groupby('customer_id')['amount'] \
                    .sum() \
                    .reset_index(name ='incoming')

most_outcoming = df_transactions \
                    .loc[df_transactions['amount'] < 0] \
                    .groupby('customer_id')['amount'] \
                    .sum() \
                    .reset_index(name ='outcoming')

# pd.set_option('display.float_format', lambda x: '%.5f' % x)
most_incoming_client, most_incoming_sum = most_incoming.loc[most_incoming['incoming'].argmax()]
most_outcoming_client, most_outcoming_sum = most_outcoming.loc[most_incoming['incoming'].argmin()]
print("Клиент с максимальной суммой приходов на карту: " + str(most_incoming_client))
print("Клиент с максимальной суммой расходов по карте: " + str(most_outcoming_client))
print(f"Модуль разницы: {most_incoming_sum - most_outcoming_sum}")
# ,most_incoming, most_outcoming

Клиент с максимальной суммой приходов на карту: 70780820.0
Клиент с максимальной суммой расходов по карте: 68273832.0
Модуль разницы: 1288934303.530022


## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [ ]:
print("Тип транзакции | Среднее значение | Медианное значение")
for tr_type in top_10:
  print(f"{tr_type} \t|\t \
  {df_transactions.loc[df_transactions['tr_type'] == tr_type]['amount'].mean()} \t|\t \
  {df_transactions.loc[df_transactions['tr_type'] == tr_type]['amount'].median()}")

print("\nСреднее значение по amount у +клиента: " + \
      str(df_transactions.loc[df_transactions['customer_id'] == most_incoming_client]['amount'].mean()) + \
      "\nМедианное значение по amount у +клиента: " + \
      str(df_transactions.loc[df_transactions['customer_id'] == most_incoming_client]['amount'].median()))
print("\nСреднее значение по amount у -клиента: " + \
      str(df_transactions.loc[df_transactions['customer_id'] == most_outcoming_client]['amount'].mean()) + \
      "\nМедианное значение по amount у -клиента: " + \
      str(df_transactions.loc[df_transactions['customer_id'] == most_outcoming_client]['amount'].median()))

Тип транзакции | Среднее значение | Медианное значение
1010 	|	   -19784.748639648507 	|	   -7411.52
2010 	|	   -136077.62932505234 	|	   -44918.32
7070 	|	   65569.83170018934 	|	   11319.42
1110 	|	   -32119.330370770458 	|	   -11207.57
1030 	|	   -5320.980221811316 	|	   -2245.92
2370 	|	   -205418.24903167944 	|	   -44918.32
7010 	|	   276391.7895961331 	|	   112295.79
7030 	|	   86104.33290896451 	|	   13951.52
7071 	|	   66806.82662251407 	|	   3593.47
1100 	|	   -44061.82726156657 	|	   -10188.259999999998

Среднее значение по amount у +клиента: -20.694945765418275
Медианное значение по amount у +клиента: 8803.99

Среднее значение по amount у -клиента: -44572.33801408431
Медианное значение по amount у -клиента: -14927.03


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [ ]:
transactions = pd.merge(df_transactions, df_gender, how='left')
transactions = pd.merge(transactions, df_mcc_codes, how='inner')
transactions = pd.merge(transactions, df_types, how='inner')
transactions.shape

(999584, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том, 
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [ ]:
print(abs(transactions.loc[transactions['gender'] == 0].loc[df_transactions['amount'] < 0]['amount'].mean()) - \
abs(transactions.loc[transactions['gender'] == 1].loc[df_transactions['amount'] < 0]['amount'].mean()))

print(abs(transactions.loc[transactions['gender'] == 0].loc[df_transactions['amount'] > 0]['amount'].mean()) - \
abs(transactions.loc[transactions['gender'] == 1].loc[df_transactions['amount'] > 0]['amount'].mean()))

-3803.8826275945357
1080.109270502995


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income". 
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [ ]:
gender_0_mean = transactions.loc[transactions['gender'] == 0].loc[transactions['amount'] > 0].groupby('tr_type')['amount'].mean().reset_index(name ='max_income')
gender_1_mean = transactions.loc[transactions['gender'] == 1].loc[transactions['amount'] > 0].groupby('tr_type')['amount'].mean().reset_index(name ='max_income')

gender_0_mean = gender_0_mean.sort_values('max_income')[:10]
gender_1_mean = gender_1_mean.sort_values('max_income')[:10]

print(gender_0_mean)
print(gender_1_mean)

print(pd.merge(gender_0_mean, gender_1_mean, on=['tr_type']))


    tr_type  max_income
10     4051  1122.96000
13     4210  2245.92000
12     4110  2245.92000
9      2370  2335.75000
11     4100  5041.86000
36     7075  6737.75000
0      1010 15766.33000
6      2110 22459.16000
18     6200 27212.88632
5      2010 33688.74000
    tr_type  max_income
6      2020  2245.92000
9      4110  2245.92000
0      1010 17658.51500
2      1110 36752.08342
29     7070 41528.73745
1      1100 47052.03500
14     6200 54222.33965
5      2010 56147.89750
23     7030 58463.29390
4      1210 61871.34000
   tr_type  max_income_x  max_income_y
0     4110    2245.92000    2245.92000
1     1010   15766.33000   17658.51500
2     6200   27212.88632   54222.33965
3     2010   33688.74000   56147.89750


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин. 
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

In [ ]:
gender_0_sum = transactions.loc[transactions['gender'] == 0].groupby('mcc_code')['amount'].sum().reset_index(name ='summ')
gender_1_sum = transactions.loc[transactions['gender'] == 1].groupby('mcc_code')['amount'].sum().reset_index(name ='summ')

gender_0_sum['diff'] = gender_0_sum['summ'] - gender_1_sum['summ']

df_diff = pd.merge(gender_0_sum, df_mcc_codes)

df_diff['diff'] = df_diff['diff'].abs()

print(df_diff.sort_values('diff', ascending=False)[:10].to_string())

     mcc_code              summ             diff                                         mcc_description
118      5995    -7177196.64000 6143488461.75953                                             Зоомагазины
22       4899     -426826.40000 6012281541.06969         Кабельные и другие платные телевизионные услуги
119      5999   -52682982.42000 4789155113.51895                      Плавательные бассейны — распродажа
21       4829 -3513723644.91983 3507846810.06983                                       Денежные переводы
120      6010  3461861161.82986 3404315578.62986        Финансовые институты — снятие наличности вручную
121      6011 -2645361971.10999 2638592429.49999  Финансовые институты — снятие наличности автоматически
54       5399    -8081165.94000  591868301.76000                      Различные товары общего назначения
55       5411  -577647951.57001  576805369.55001                       Бакалейные магазины, супермаркеты
63       5533   -12661398.49000  192920590.86000       

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов. 

In [ ]:
gender_0_night = transactions.loc[transactions['tr_datetime'].str.split().str[1].str[:2].astype('int32') <= 5].loc[transactions['gender'] == 0].loc[transactions['amount'] < 0]
gender_1_night = transactions.loc[transactions['tr_datetime'].str.split().str[1].str[:2].astype('int32') <= 5].loc[transactions['gender'] == 1].loc[transactions['amount'] < 0]
print(len(gender_0_night))
print(len(gender_1_night))

38827
43017
